In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium import webdriver
import os
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [8]:
class Zarascrape:

    def __init__ (self, url): # Declaro las variables self de la clase
        self.url = url 
        self.col_names = ["url", "description"]
        self.product_list = [] 
        
    def get_info(self, url): # saca la info de cada vestido en particular
        html = requests.get(url).content
        data = BeautifulSoup(html, "lxml")
        prod_title=data.find("h1",{"class":"product-name"})
        title=prod_title.text.replace(" Detalle","")
        prod_descr = data.find("p",{"class":"description"})
        prod_various = data.find("p",{"class":"product-color"})
        prod_color=prod_various.find("span",{"class": "_colorName"})
        prod_ref=prod_various.find("span",{"data-qa-qualifier": "product-reference"})
        prod_price=data.find_all("script",{"type":"application/ld+json"})
        for i in prod_price:
            price=i.text.split(',')[-2].lstrip('"price": ').replace('"','')
        prod_size=data.find_all("div",{"class":"size-select"})
        for all_size in prod_size:
            size=all_size.find_all("label", {'class':'product-size _product-size'}) #filtramos por tallas disponibles
            available_size=[s.text for s in size]
        prod_images=data.find_all("a",{"class":"_seoImg main-image"})
        prod_image=[e.get("href") for e in prod_images][0].lstrip("//")
        try:
            features=[title,prod_descr.text,prod_color.text,prod_ref.text, price, available_size,prod_image]
        except:
            features=["","","","","","",""]
        return features
    
    def get_attrs(self,webelement, attr='href'): 
        try: 
            return webelement.get_attribute(attr)
        except: 
            return ""
    
    def get_list(self): # List of urls para vestidos
        driver = webdriver.Chrome((ChromeDriverManager().install()))
        driver.get(self.url)
        source_data = driver.page_source
        elem = driver.find_element_by_tag_name("body")

        no_of_pagedowns = 10

        res = []

        while no_of_pagedowns:
            print(no_of_pagedowns)
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            no_of_pagedowns-=1
        div = driver.find_elements_by_class_name('product-info-item')
        for i in div:
            b=i.find_elements_by_css_selector('a')
            for c in b:
                res.append(c)
        return [self.get_attrs(e, 'href') for e in res]

    def loop_it(self): # Iterates over the url root. 
        iterable = self.get_list()
        for e in iterable:
            print (len(self.product_list))
            self.product_list.append(self.get_info(e))
            time.sleep(3)
        return self.product_list
    
url_vestidos = 'https://www.zara.com/uk/en/woman-dresses-l1066.html?v1=1180427' # Scrapea sólo vestidos

# Assigns the Zarascrape class for dresses to an object
vestidos_ls = Zarascrape(url_vestidos)

# Launches the class method
data_vestidos = vestidos_ls.loop_it()

print(len(data_vestidos))
data_vestidos[:5]


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/almu/.wdm/chromedriver/2.46/mac64/chromedriver
10
9
8
7
6
5
4
3
2
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242


[['POLKA DOT TULLE DRESS Details',
  'Semi-sheer collared dress with long sleeves and a buttoned front.MODEL HEIGHT: 177 CM',
  'Ecru',
  '4437/069',
  '39.99',
  [],
  'static.zara.net/photos///2019/V/0/1/p/4437/069/712/2/w/560/4437069712_1_1_1.jpg?ts=1551896482917'],
 ['LACE DRESS Details',
  'Short sleeve dress with a shirt collar. Featuring matching interior lining and metal button fastening in the front. HEIGHT OF MODEL: 177 cm. / 5′ 9″',
  'White',
  '1639/056',
  '39.99',
  ['XS', 'S', 'M'],
  'static.zara.net/photos///2019/V/0/1/p/1639/056/250/2/w/560/1639056250_1_1_1.jpg?ts=1552560925886'],
 ['LONG POLKA DOT DRESS Details',
  'Collared dress with long sleeves. Featuring an A-line silhouette, a matching belt detail and front button fastening.HEIGHT OF MODEL: 177 CM / 5′ 9″',
  'Off-white',
  '3440/055',
  '29.99',
  ['XS', 'S', 'M', 'L', 'XL', 'XXL'],
  'static.zara.net/photos///2019/V/0/1/p/3440/055/251/3/w/560/3440055251_1_1_1.jpg?ts=1552919570182'],
 ['DRESS WITH MATCHING EM

In [9]:
# Saves the df into a .csv
zara_base = pd.DataFrame(data_vestidos)

In [10]:
zara_base.head()

,0,1,2,3,4,5,6
0,POLKA DOT TULLE DRESS Details,Semi-sheer collared dress with long sleeves an...,Ecru,4437/069,39.99,[],static.zara.net/photos///2019/V/0/1/p/4437/069...
1,LACE DRESS Details,Short sleeve dress with a shirt collar. Featur...,White,1639/056,39.99,"[XS, S, M]",static.zara.net/photos///2019/V/0/1/p/1639/056...
2,LONG POLKA DOT DRESS Details,Collared dress with long sleeves. Featuring an...,Off-white,3440/055,29.99,"[XS, S, M, L, XL, XXL]",static.zara.net/photos///2019/V/0/1/p/3440/055...
3,DRESS WITH MATCHING EMBROIDERY Details,V-neck dress with sleeves falling below the el...,White,1821/021,25.99,"[XS, S, M, L]",static.zara.net/photos///2019/V/0/1/p/1821/021...
4,EMBROIDERED LACE DRESS Details,Round neck dress with sleeves reaching below t...,Ecru,5770/022,29.99,"[XS, S, M, L]",static.zara.net/photos///2019/V/0/1/p/5770/022...


In [11]:
zara_base.shape

(641, 7)

In [12]:
# Saves the df into a .csv
zara_dresses = zara_base.to_csv("zara2.csv", index=False)